In [111]:
import os
import json
import random
from pathlib import Path

from dotenv import load_dotenv
from datasets import Dataset, concatenate_datasets
import pandas as pd

print("Loaded .env:", load_dotenv("../../.env", override=True))
data_dir = Path(os.environ["PROJECT_DIR"]) / "data" / "host"
preds_dir = Path(os.environ["PROJECT_DIR"]) / "predict" / "mini_v1_df"

LANG = "es"
SPLIT = 'test'

VAL_SCORES = {}

OUTPUT_DIR = data_dir.parent / "subs" / "v1"

Loaded .env: True


In [112]:
import json
from datasets import Dataset

dip_t1 = pd.DataFrame(json.load(open(data_dir / "dipromats_2023" / f"{SPLIT}_t1_{LANG}.json")))
dip_t2 = pd.DataFrame(json.load(open(data_dir / "dipromats_2023" / f"{SPLIT}_t2_{LANG}.json")))
dip_t3 = pd.DataFrame(json.load(open(data_dir / "dipromats_2023" / f"{SPLIT}_t3_{LANG}.json")))
dip_pred_df = pd.read_parquet(preds_dir / f"dipromats_2023_{SPLIT}_t3_{LANG}_preds_df.parquet")

In [113]:
dip_pred_df

,t1_pred,t2_pred,t3_pred,gt
0,true,[2 discrediting the opponent],[2 discrediting the opponent - name calling],
1,false,[false],[false],
2,false,[false],[false],
3,false,[false],[false],
4,false,[false],[false],
...,...,...,...,...
3465,false,[false],[false],
3466,false,[false],[false],
3467,false,[false],[false],
3468,false,[false],[false],


In [114]:
dip_t1

,test_case,id,text
0,DIPROMATS2023,6120,1/2 James Story: “El informe realizado por la ...
1,DIPROMATS2023,6121,Ayer participé de un fructífero intercambio de...
2,DIPROMATS2023,6122,🇪🇺 La Unión Europea nació como un proyecto de ...
3,DIPROMATS2023,6123,La presidenta @vonderleyen pronunció su primer...
4,DIPROMATS2023,6124,"Según un informe de la ONU de esa semana, el r..."
...,...,...,...
3465,DIPROMATS2023,9586,Tras su mensaje durante el foro virtual Think ...
3466,DIPROMATS2023,9587,El Parlamento declara la Unión Europea «zona d...
3467,DIPROMATS2023,9588,Día completo: donamos kits de bioseguridad par...
3468,DIPROMATS2023,9589,En la #UE 🇪🇺 impulsamos políticas en favor de ...


In [115]:
set(dip_t2.id) - set(dip_t3.id)

set()

In [116]:
assert (dip_t1.id != dip_t2.id).sum() == 0
assert (dip_t1.id != dip_t3.id).sum() == 0

In [121]:
dip_t1_sub = [
    {
        "test_case": "DIPROMATS2023",
        "id": id_,
        "value": value
    }
    for id_, value in zip(dip_t1.id, dip_pred_df.t1_pred)
]

if SPLIT == "test":
    json.dump(dip_t1_sub, open(OUTPUT_DIR / f"DIPROMATS_2023_T1_{LANG}.json", "w"))

In [118]:
dip_t1["pred"] = dip_pred_df.t1_pred
if SPLIT == "val":
    (dip_t1["value"] == dip_t1["pred"]).mean()

In [122]:
dip_t2["pred"] = dip_pred_df.t2_pred

display(dip_t2)

dip_t2_sub = [
    {
        "test_case": "DIPROMATS2023",
        "id": id_,
        "value": value.tolist()
    }
    for id_, value in zip(dip_t2.id, dip_pred_df.t2_pred)
]

if SPLIT == "test":
    json.dump(dip_t2_sub, open(OUTPUT_DIR / f"DIPROMATS_2023_T2_{LANG}.json", "w"))

random.choice(dip_t2_sub)

,test_case,id,text,pred
0,DIPROMATS2023,6120,1/2 James Story: “El informe realizado por la ...,[2 discrediting the opponent]
1,DIPROMATS2023,6121,Ayer participé de un fructífero intercambio de...,[false]
2,DIPROMATS2023,6122,🇪🇺 La Unión Europea nació como un proyecto de ...,[false]
3,DIPROMATS2023,6123,La presidenta @vonderleyen pronunció su primer...,[false]
4,DIPROMATS2023,6124,"Según un informe de la ONU de esa semana, el r...",[false]
...,...,...,...,...
3465,DIPROMATS2023,9586,Tras su mensaje durante el foro virtual Think ...,[false]
3466,DIPROMATS2023,9587,El Parlamento declara la Unión Europea «zona d...,[false]
3467,DIPROMATS2023,9588,Día completo: donamos kits de bioseguridad par...,[false]
3468,DIPROMATS2023,9589,En la #UE 🇪🇺 impulsamos políticas en favor de ...,[false]


{'test_case': 'DIPROMATS2023', 'id': '9431', 'value': ['false']}

In [120]:
dip_t3["pred"] = dip_pred_df.t3_pred

display(dip_t3)

dip_t3_sub = [
    {
        "test_case": "DIPROMATS2023",
        "id": id_,
        "value": value.tolist()
    }
    for id_, value in zip(dip_t3.id, dip_pred_df.t3_pred)
]

if SPLIT == "test":
    json.dump(dip_t3_sub, open(OUTPUT_DIR / f"DIPROMATS_2023_T3_{LANG}.json", "w"))

random.choice(dip_t3_sub)

,test_case,id,text,pred
0,DIPROMATS2023,6120,1/2 James Story: “El informe realizado por la ...,[2 discrediting the opponent - name calling]
1,DIPROMATS2023,6121,Ayer participé de un fructífero intercambio de...,[false]
2,DIPROMATS2023,6122,🇪🇺 La Unión Europea nació como un proyecto de ...,[false]
3,DIPROMATS2023,6123,La presidenta @vonderleyen pronunció su primer...,[false]
4,DIPROMATS2023,6124,"Según un informe de la ONU de esa semana, el r...",[false]
...,...,...,...,...
3465,DIPROMATS2023,9586,Tras su mensaje durante el foro virtual Think ...,[false]
3466,DIPROMATS2023,9587,El Parlamento declara la Unión Europea «zona d...,[false]
3467,DIPROMATS2023,9588,Día completo: donamos kits de bioseguridad par...,[false]
3468,DIPROMATS2023,9589,En la #UE 🇪🇺 impulsamos políticas en favor de ...,[false]


{'test_case': 'DIPROMATS2023', 'id': '8960', 'value': ['false']}

In [75]:
exist_2022_t1 = pd.DataFrame(json.load(open(data_dir / "exist_2022" / f"{SPLIT}_t1_{LANG}.json")))
exist_2022_t2 = pd.DataFrame(json.load(open(data_dir / "exist_2022" / f"{SPLIT}_t2_{LANG}.json")))
exist_pred_df = pd.read_parquet(preds_dir / f"exist_2022_{SPLIT}_t2_{LANG}_preds_df.parquet")

In [76]:
exist_2022_t1["pred"] = exist_pred_df.t1_pred
exist_2022_t2["pred"] = exist_pred_df.t2_pred

display(exist_2022_t1)
display(exist_2022_t2)

,test_case,id,source,language,text,value,pred
0,EXIST2021,8057,twitter,en,@ProfSlakes @lesiba_thabetha @errolbsk @norman...,sexist,non-sexist
1,EXIST2021,8058,twitter,en,Equality act passes in US House; leads to conf...,non-sexist,non-sexist
2,EXIST2021,8059,twitter,en,Isaias Afewerki &amp; @AbiyAhmedAli both lied ...,non-sexist,non-sexist
3,EXIST2021,8060,twitter,en,Fellas make women feel safe walking by again 2...,non-sexist,non-sexist
4,EXIST2021,8061,gab,en,gov't destroyed black families with welfare.fe...,sexist,sexist
...,...,...,...,...,...,...,...
1124,EXIST2021,9181,twitter,en,@Hazel08539106 @hatterme123 @lilhazelnuttt You...,non-sexist,non-sexist
1125,EXIST2021,9182,twitter,en,"how are you, a grown ass woman that’s literall...",non-sexist,sexist
1126,EXIST2021,9183,gab,en,Another gold digger looking to make money off ...,sexist,non-sexist
1127,EXIST2021,9184,twitter,en,We have another first!! So glad to see women d...,non-sexist,non-sexist


,test_case,id,source,language,text,value,pred
0,EXIST2021,8057,twitter,en,@ProfSlakes @lesiba_thabetha @errolbsk @norman...,misogyny-non-sexual-violence,non-sexist
1,EXIST2021,8058,twitter,en,Equality act passes in US House; leads to conf...,non-sexist,non-sexist
2,EXIST2021,8059,twitter,en,Isaias Afewerki &amp; @AbiyAhmedAli both lied ...,non-sexist,non-sexist
3,EXIST2021,8060,twitter,en,Fellas make women feel safe walking by again 2...,non-sexist,non-sexist
4,EXIST2021,8061,gab,en,gov't destroyed black families with welfare.fe...,ideological-inequality,ideological-inequality
...,...,...,...,...,...,...,...
1124,EXIST2021,9181,twitter,en,@Hazel08539106 @hatterme123 @lilhazelnuttt You...,non-sexist,non-sexist
1125,EXIST2021,9182,twitter,en,"how are you, a grown ass woman that’s literall...",non-sexist,non-sexist
1126,EXIST2021,9183,gab,en,Another gold digger looking to make money off ...,stereotyping-dominance,non-sexist
1127,EXIST2021,9184,twitter,en,We have another first!! So glad to see women d...,non-sexist,non-sexist


In [77]:
if SPLIT == "val":
    VAL_SCORES[f"exist_2022_t1_{LANG}"] = (exist_2022_t1["value"] == exist_2022_t1["pred"]).mean()

In [78]:
from sklearn.metrics import f1_score
if SPLIT == "val":
    VAL_SCORES[f"exist_2022_t2_{LANG}"] = f1_score(exist_2022_t2["value"], exist_2022_t2["pred"], average="macro")

In [90]:
exist_2022_t1_sub = [
    {
        "test_case": "EXIST2022",
        "id": id_,
        "value": value
    }
    for id_, value in zip(exist_2022_t1.id, exist_2022_t1["pred"])
]

if SPLIT == "test":
    json.dump(exist_2022_t1_sub, open(OUTPUT_DIR / f"EXIST_2022_T1_{LANG}_sub.json", "w"))

exist_2022_t2_sub = [
    {
        "test_case": "EXIST2022",
        "id": id_,
        "value": value
    }
    for id_, value in zip(exist_2022_t2.id, exist_2022_t2["pred"])
]

if SPLIT == "test":
    json.dump(exist_2022_t2_sub, open(OUTPUT_DIR / f"EXIST_2022_T2_{LANG}.json", "w"))

random.choice(exist_2022_t2_sub)


{'test_case': 'EXIST2022', 'id': '8974', 'value': 'sexual-violence'}

{
'exist_2022_t1_en': 0.7856510186005314,
 'exist_2022_t2_en': 0.5530430494882652,
 'exist_2022_t1_es': 0.85977212971078, 
 'exist_2022_t2_es': 0.7054053952583365
 'sqac_t1_en': 0.4179503825072858,
 'sqac_t1_es': 0.546566238535548
}


In [108]:
sqac_t1 = pd.DataFrame(json.load(open(data_dir / "sqac_squad_2024" / f"{SPLIT}_t1_{LANG}.json")))
sqac_pred_df = pd.read_parquet(preds_dir / f"sqac_squad_2024_{SPLIT}_t1_{LANG}_preds_df.parquet")
sqac_t1["pred"] = sqac_pred_df.pred

display(sqac_t1)

,id,title,context,question,test_case,value,pred
0,61b68255-ac90-4a09-8158-55f94342f442,Lenguas de México,El término lenguas de México se refiere a los ...,¿Cuántas lenguas aborígenes existen en México?,SQAC_SQUAD_2024,al menos 68,al menos 68
1,06867cbd-62f1-49ce-883c-4d4c0e223f4e,Lenguas de México,El término lenguas de México se refiere a los ...,¿Cuál es el idioma más hablado en México?,SQAC_SQUAD_2024,español,español
2,60eafcf7-2e7d-47ad-9ac9-3cce12611f70,Lenguas de México,El término lenguas de México se refiere a los ...,¿Qué lenguajes son considerados lenguas nacion...,SQAC_SQUAD_2024,las lenguas indígenas y el español,las lenguas indígenas y el español
3,0bf46c7e-3f5e-4f07-9f1b-031a799ee66f,Lenguas de México,Variedades mexicanas del español \nLos diverso...,¿Con qué intención se han estudiado los dialec...,SQAC_SQUAD_2024,con el propósito de comprender sus característ...,con el propósito de comprender sus característ...
4,91bd3446-3e79-4e14-b620-23c2b0b3b99f,Lenguas de México,Variedades mexicanas del español \nLos diverso...,¿Qué zonas abarca la primera rama del español ...,SQAC_SQUAD_2024,"norte, centro y zonas costeras de México","las hablas del norte, centro y zonas costeras ..."
...,...,...,...,...,...,...,...
1859,656f2733-9241-4b52-b406-1334dfe9860a,Irán ignora demandas de transparencia por part...,16 de septiembre de 2008 Tras el pedido de tra...,¿Qué piensa la AIEA que pretende construir Irán?,SQAC_SQUAD_2024,una bomba atómica,una bomba atómica
1860,3f7b6ad7-5086-4e62-94ee-957fd5100133,Presidente de la Cámara de los Diputados de Br...,25 de septiembre de 2005 El Presidente de la...,¿Qué puesto ocupaba Cavalcanti hasta el moment...,SQAC_SQUAD_2024,Presidente de la Cámara de Diputados de Brasil,Presidente de la Cámara de Diputados de Brasil
1861,3bf11b74-8efc-4e5d-8a44-dde308fe4da8,Presidente de la Cámara de los Diputados de Br...,25 de septiembre de 2005 El Presidente de la...,¿En qué falta podría haber incurrido Cavalcant...,SQAC_SQUAD_2024,quiebra de decoro parlamentaria,quiebra de decoro parlamentaria
1862,066bac61-ad40-41e0-b37a-4bf65fbf7fd1,Dovizioso vence en el penúltimo GP de la tempo...,2 de noviembre de 2017 El mundial está llegand...,¿Qué pilotos rivalizan por el primer puesto en...,SQAC_SQUAD_2024,Andrea Dovizioso y Marc Márquez,Andrea Dovizioso y Marc Márquez


In [104]:
sqac_t1_sub = [
    {
        "test_case": "SQAC_SQUAD_2024",
        "id": id_,
        "value": value
    }
    for id_, value in zip(sqac_t1.id, sqac_t1["pred"])
]

if SPLIT == "test":
    json.dump(sqac_t1_sub, open(OUTPUT_DIR / f"SQAC_SQUAD_2024_{LANG}.json", "w"))


In [109]:
if SPLIT == "val":
    VAL_SCORES[f"sqac_t1_{LANG}"] = f1_score(sqac_t1["value"], sqac_t1["pred"], average="macro")

In [110]:
VAL_SCORES

{'sqac_t1_es': 0.546566238535548}

In [71]:
VAL_SCORES

{'exist_2022_t1_es': 0.85977212971078, 'exist_2022_t2_es': 0.7054053952583365}